## Association Rules
Regels die verbanden weer geven tussen bepaalde zaken wat kopen we vaak samen?

### Transformatie
We moeten er voor zorgen dat we het juiste formaat aanleveren

#### Lijst van lijst
 * Gebruik een TransactionEncoder wanneer je gegevens in dit formaat staan

In [2]:
#FOUT FORMAAT
transacties = [['Printer', 'Cartridge', 'Balpen'],
                ['Papier', 'Cartridge'],
                ['Papier', 'Cartridge'],
                ['Printer', 'Cartridge'],
                ['Printer', 'Cartridge'],
                ['Balpen'],
                ['Papier', 'Cartridge', 'Balpen'],
                ['Cartridge'],
                ['Printer', 'Balpen'],
                ['Papier']]
transacties

[['Printer', 'Cartridge', 'Balpen'],
 ['Papier', 'Cartridge'],
 ['Papier', 'Cartridge'],
 ['Printer', 'Cartridge'],
 ['Printer', 'Cartridge'],
 ['Balpen'],
 ['Papier', 'Cartridge', 'Balpen'],
 ['Cartridge'],
 ['Printer', 'Balpen'],
 ['Papier']]

In [4]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
dataset = te.fit(transacties).transform(transacties)
# combineer in Dataframe voor een correct formaat
kassatickets = pd.DataFrame(dataset, columns=te.columns_, index=range(1000123,1000133))
kassatickets


,0,1,2,3
1000123,True,True,False,True
1000124,False,True,True,False
1000125,False,True,True,False
1000126,False,True,False,True
1000127,False,True,False,True
1000128,True,False,False,False
1000129,True,True,True,False
1000130,False,True,False,False
1000131,True,False,False,True
1000132,False,False,True,False


#### Sequentieel formaat (ALLES ONDER ELKAAR)

* Gebruik een get_dummies wanneer je gegevens in dit formaat staan


In [78]:
#FOUT FORMAAT
import pandas as pd
transacties = pd.read_csv('./transacties.csv',sep=';', index_col=0)
transacties

,Product
KassaTicket,
1000123,Printer
1000123,Cartridge
1000123,Balpen
1000124,Papier
1000124,Cartridge
1000125,Papier
1000125,Cartridge
1000126,Printer
1000126,Cartridge


In [79]:
kassatickets = pd.get_dummies(transacties, prefix='', prefix_sep='').groupby(level='KassaTicket').sum()
kassatickets


,Balpen,Cartridge,Papier,Printer
KassaTicket,,,,
1000123,1,1,0,1
1000124,0,1,1,0
1000125,0,1,1,0
1000126,0,1,0,1
1000127,0,1,0,1
1000128,1,0,0,0
1000129,1,1,1,0
1000130,0,1,0,0
1000131,1,0,0,1


#### Support
Het aantal keer dat een product A voor komt gedeeld door het het totaal van dat product.<br>
De kans dat je dat product vindt in een winkelkar.
De **onvoorwaardleijke kans** **PRIOR KANS**


$Support(A)=\frac{\#A}{N}$
$Support(Printer)=\frac{4}{10}=0.4$




### Algortime 1

#### Apriori
Legt een miminum support op. ( bv van 0.2 )
Als er ergens een comibnatie is die onder de 0.2 valt dan wordt deze geschrapt.
Alle de mogelijk combinaties worden gemaakt en er wordt gekeken hoe vaak deze komen deze voorkomen.



In [61]:
# transacties.csv

import pandas as pd
from mlxtend.frequent_patterns import fpgrowth
transacties1 = pd.read_csv('transacties.csv',sep=';', index_col=0)
transacties1



# FROMAAT1

kassatickets1 = pd.get_dummies(transacties1, prefix='', prefix_sep='').groupby('KassaTicket').sum()
display(kassatickets1)
supports = kassatickets1.mean(axis=0)
supports



,Balpen,Cartridge,Papier,Printer
KassaTicket,,,,
1000123,1,1,0,1
1000124,0,1,1,0
1000125,0,1,1,0
1000126,0,1,0,1
1000127,0,1,0,1
1000128,1,0,0,0
1000129,1,1,1,0
1000130,0,1,0,0
1000131,1,0,0,1


Balpen       0.4
Cartridge    0.7
Papier       0.4
Printer      0.4
dtype: float64

In [62]:
# FROMAAT2
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

transacties2 = [['Printer', 'Cartridge', 'Balpen'],
                ['Papier', 'Cartridge'],
                ['Papier', 'Cartridge'],
                ['Printer', 'Cartridge'],
                ['Printer', 'Cartridge'],
                ['Balpen'],
                ['Papier', 'Cartridge', 'Balpen'],
                ['Cartridge'],
                ['Printer', 'Balpen'],
                ['Papier']]




te = TransactionEncoder()
dataset = te.fit(transacties2).transform(transacties2)
# combineer in Dataframe voor een correct formaat
kassatickets2 = pd.DataFrame(dataset, columns=te.columns_, index=range(1000123,1000133))
kassatickets2

display(kassatickets2)

supports = kassatickets2.mean(axis=0)
supports

,Balpen,Cartridge,Papier,Printer
1000123,True,True,False,True
1000124,False,True,True,False
1000125,False,True,True,False
1000126,False,True,False,True
1000127,False,True,False,True
1000128,True,False,False,False
1000129,True,True,True,False
1000130,False,True,False,False
1000131,True,False,False,True
1000132,False,False,True,False


Balpen       0.4
Cartridge    0.7
Papier       0.4
Printer      0.4
dtype: float64

In [65]:
from mlxtend.frequent_patterns import apriori
itemsets = apriori(kassatickets1, min_support=0.3, use_colnames=True)
itemsets


,support,itemsets
0,0.4,(Balpen)
1,0.7,(Cartridge)
2,0.4,(Papier)
3,0.4,(Printer)
4,0.2,"(Cartridge, Balpen)"
5,0.1,"(Papier, Balpen)"
6,0.2,"(Balpen, Printer)"
7,0.3,"(Cartridge, Papier)"
8,0.3,"(Cartridge, Printer)"
9,0.1,"(Cartridge, Papier, Balpen)"


In [64]:
from mlxtend.frequent_patterns import apriori
itemsets = apriori(kassatickets2, min_support=0.3, use_colnames=True)
itemsets



,support,itemsets
0,0.4,(Balpen)
1,0.7,(Cartridge)
2,0.4,(Papier)
3,0.4,(Printer)
4,0.2,"(Cartridge, Balpen)"
5,0.1,"(Papier, Balpen)"
6,0.2,"(Balpen, Printer)"
7,0.3,"(Cartridge, Papier)"
8,0.3,"(Cartridge, Printer)"
9,0.1,"(Cartridge, Papier, Balpen)"





### Algortime 2

#### FP Growth efficienter als Apriori
* 1. Tel de frequenties van alle items.<br><i>**Doorloop de transacties een eerste keer**</i>

* 2. Schrap de items met een support kleiner dan minimumsupport

* 3. Sorteer de items van groot naar klein volgens support

* 4. Bouw transactie per transactie een boomstructuur op<br><i>Doorloop de transacties een tweede keer</i>

* 5. Extraheer de supports uit de boomstructuur


In [69]:
# FORMAAT 1
from mlxtend.frequent_patterns import fpgrowth
itemsets = fpgrowth(kassatickets1, min_support = 0.3,
use_colnames=True)
itemsets

,support,itemsets
0,0.7,(Cartridge)
1,0.4,(Printer)
2,0.4,(Balpen)
3,0.4,(Papier)
4,0.3,"(Cartridge, Printer)"
5,0.3,"(Cartridge, Papier)"


In [70]:
# FORMAAT 2
from mlxtend.frequent_patterns import fpgrowth
itemsets = fpgrowth(kassatickets2, min_support = 0.3,
use_colnames=True)
itemsets



,support,itemsets
0,0.7,(Cartridge)
1,0.4,(Printer)
2,0.4,(Balpen)
3,0.4,(Papier)
4,0.3,"(Cartridge, Printer)"
5,0.3,"(Cartridge, Papier)"


### Alles samen

In [85]:
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth, association_rules
# stap 1 - datamanagement
kassatickets = pd.read_csv('transacties.csv', sep=';', index_col=0)
transacties = pd.get_dummies(kassatickets, columns=['Product'],
prefix='',prefix_sep='').groupby(level='KassaTicket').sum()
# stap 2 - itemsets met support berekenen
itemsets = fpgrowth(transacties, min_support=0.3, use_colnames=True)
# stap 3 - association rules zoeken
rules = association_rules(itemsets, min_threshold=0.5)

itemsets
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Printer),(Cartridge),0.4,0.7,0.3,0.75,1.071429,0.02,1.2
1,(Papier),(Cartridge),0.4,0.7,0.3,0.75,1.071429,0.02,1.2
